In [ ]:
import pandas as pd

In [7]:
pheno = pd.read_csv('../16p12.2_rnaseq_analysis/data/pheno_final.tsv', sep='\t')

subs = list(pheno.subject.unique())

sub2samp = {}
for sub in subs:
    samps = pheno[pheno.subject == sub]['samp'].to_list()
    sub2samp[sub] = samps

In [37]:
def reformat_header(header):
    header = header.strip().split('\t')
    new_header = []
    
    for col in header:
        if not col.startswith('PS'):
            new_header.append(col)
            continue
        fam = col.split('_')[0]
        sub = col.split('_')[1]
        for samp in sub2samp[sub]:
            new_col = '{}{}_{}'.format(fam, sub, samp)
            new_header.append(new_col)
    
    new_header = '\t'.join(new_header) + '\n'
    return new_header

def reformat_line(s):
    old_line = s.strip().split('\t')
    new_line = []
    
    for i in range(len(old_line)):
        item = old_line[i]
        
        if i < 9:
            new_line.append(item)
            continue
        
        for j in range(3):
            new_line.append(item)
    
    new_line = '\t'.join(new_line) + '\n'
    return new_line

In [39]:
vcfin = 'plink_files/16p12_RNA_vcf.vcf'
vcfout = '16p12_RNA_by_replicate.vcf'

fin = open(vcfin, 'r')
fout = open(vcfout, 'w')

ln = 0
for l in fin:
    if l.startswith('##'):
        fout.write(l)
        continue
        
    if l.startswith('#CHROM'):
        new_l = reformat_header(l)
        fout.write(new_l)
        continue
    
    new_l = reformat_line(l)
    fout.write(new_l)
    
#     if ln > 50:
#         break
#     ln = ln + 1


fin.close()
fout.close()